In [1]:
import os
import rasterio
import numpy as np

import shutil 
from tqdm import tqdm
import numpy as np

In [2]:
# Path to the directory containing the GeoTIFF files
orig_directory_path = r'P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\02_Exposure\Elevation_DEM\copDEM_Remove_Empty_20m'
clip_output_path=r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\02_Exposure\Elevation_DEM\copDEM_Remove_Empty_10m"

len(os.listdir(orig_directory_path))

8624

# Clipping the elevation data of each tiff between 10m & 0m
## This helps to reduce the number of polygons created ie less space and computation time
## All pixels outside of this number are given -9999 ie ""no value"
## Empty tiffs are however still present and should be reduced in the next step

In [3]:


# Loop over each file in the directory
for filename in tqdm(os.listdir(orig_directory_path),desc='Clipping files'):
    if filename.endswith('.tif'):
        file_path = os.path.join(orig_directory_path, filename)
        
        # Open the GeoTIFF file
        with rasterio.open(file_path) as src:
            # Read the raster data into a NumPy array
            arr = src.read(1)
            
            # Replace values greater than 10 with -9999
            arr = np.where( (arr > 10) | (arr<0), -9999, arr)
            
            # Create a new filename by appending '_below_10m' to the original filename
            new_filename = os.path.splitext(str(filename))[0] + '_below_10m.tif'
            new_file_path = os.path.join(clip_output_path, new_filename)
            
            # Write the modified raster data to the new GeoTIFF file
            with rasterio.open(new_file_path, 'w', **src.profile) as dst:
                dst.write(arr, 1)



Clipping files: 100%|██████████| 8624/8624 [58:12<00:00,  2.47it/s]  


# The above method export all the imported files even if they are empty ie all pixels == -9999
## Remove empty features from the list and save them in a new file
## this helps to reduce the number of files to be polygonized in the next step

In [4]:


source_folder = clip_output_path
empty_remove_destination_folder = r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\02_Exposure\Elevation_DEM\copDEM_Remove_Empty_10m_clip"


# Get list of all .tiff files in the source folder
tiff_files = [f for f in os.listdir(source_folder) if f.endswith('.tif')]

# Add a progress bar
for file in tqdm(tiff_files, desc='Copying files'):


    file_path = os.path.join(source_folder, file)
    
    with rasterio.open(file_path) as src:
        # Check if all pixel values are -9999
        if (-9999 == src.read().ravel()).all():
            continue # Skip this file and move to next one
        
        # Copy the file to the destination folder
        dest_path = os.path.join(empty_remove_destination_folder, file)
        shutil.copy2(file_path, dest_path)


Copying files: 100%|██████████| 8624/8624 [40:39<00:00,  3.54it/s]  
